In [1]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk import FreqDist
import re
#import en_core_web_sm
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import numpy as np #For divergences/distances
import scipy as sp#For divergences/distances
import sklearn.manifold #For a manifold plot
import urllib.parse #For joining urls
import matplotlib.colors
import graphviz 
import sklearn
import sklearn.naive_bayes
import sklearn.tree
import sklearn.ensemble
import sklearn.neural_network
import sklearn.decomposition
import collections
import os.path
import random
import glob
import requests
import json
import math
import networkx as nx
import logging
import pickle #if you want to save layouts

%matplotlib inline
import en_core_web_sm
nlp = en_core_web_sm.load()
#from textblob import TextBlob
import sys
#import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nltk
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image

from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
plt.style.use('fivethirtyeight')
import matplotlib.ticker as mticker
from scipy import stats
import sklearn.feature_extraction
from networkx.algorithms.community import greedy_modularity_communities

import emoji

#Courtesy of Computational-Content-Analysis-2020. This package is not up-to-date at the link and will take some alteration locally
import lucem_illud_2020 #pip install -U git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git


import random
import pandas as pd
import numpy as np
from multiprocessing import  Pool

import pandas as pd
import numpy as np

import re
import string
import os
import spacy

import gensim


# libraries for visualization

import matplotlib.pyplot as plt
import seaborn as sns




import random




from gensim import corpora
from datetime import datetime
from gensim.test.utils import datapath

from gensim.models.ldamulticore import LdaMulticore
from gensim.corpora import Dictionary
from gensim.test.utils import datapath

In [2]:
files = [#'Jan2020',
#'Apr2020',
#'Feb2021',
#'Jun2020',
#'Nov2020',
#'May2020',
#'Mar2021',
#'Apr2021',
#'Oct2020',
#'Jul2021',
#'Sep2020',
#'Sep2021',
#'Aug2020',
#'May2021',
#'Dec2020',
#'Feb2020',
#'Aug2021',
#'Jan2021',
#'Jan2022',
#'Jun2021',
#'Feb2022',
#'Oct2021',
#'Jul2020',
#'Nov2021',
#'Mar2020',
'Dec2021',]

In [3]:
for f in files:
    

    
    print(f)
    
    comment_DF = pd.read_csv("%s.csv" %f)
    len(comment_DF)
    logger = logging.getLogger("spacy")
    logger.setLevel(logging.ERROR)
    
    def tokeandnorm(comment_DF):
        comment_DF['tokenized_sents'] = comment_DF['comment'].apply(lambda x: [lucem_illud_2020.word_tokenize(s) for s in lucem_illud_2020.sent_tokenize(x)])
        comment_DF['normalized_sents'] = comment_DF['tokenized_sents'].apply(lambda x: [lucem_illud_2020.normalizeTokens(s) for s in x])
        return comment_DF


    def parallelize_dataframe(comment_DF, func, n_cores=48):
        df_split = np.array_split(comment_DF, n_cores)
        pool = Pool(n_cores)
        comment_DF = pd.concat(pool.map(func, df_split))
        pool.close()
        pool.join()
        return comment_DF

    comment_DF = parallelize_dataframe(comment_DF, tokeandnorm)    

    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)
    
    
    #Cleaning CLEAN MORE
    def cleanTxt(text):
        
        text = re.sub(r'@[A-Za-z0-9]+', '', str(text)) #
        text = re.sub(r'#', '', text)
        text = re.sub(r'RT[\s]+', '', str(text))
        text = re.sub(r'https?:\/\/\S+', '', str(text))
        text = re.sub(r'>', '', str(text))
        text = re.sub(r'<', '', str(text))
        text = re.sub(r'=', '', str(text))
        text = re.sub(r'|', '', str(text))
        text = re.sub(r'jeffrey', '', str(text))
        text = re.sub(r'epstein', '', str(text))
        text = re.sub(r'et', '', str(text))
        text = re.sub(r'al', '', str(text))
        text =  re.sub(emoji.get_emoji_regexp(), r"", text)
        text = emoji_pattern.sub(r'', text)
        return text
    
    def cleanit(comment_DF):
        comment_DF['normalized_sents'] = comment_DF['normalized_sents'].apply(cleanTxt)
        return comment_DF

    def parallelize_dataframe(comment_DF, func, n_cores=48):
        df_split = np.array_split(comment_DF, n_cores)
        pool = Pool(n_cores)
        comment_DF = pd.concat(pool.map(func, df_split))
        pool.close()
        pool.join()
        return comment_DF
    
    comment_DF = parallelize_dataframe(comment_DF, cleanit)

    
    PosDF = comment_DF.loc[comment_DF['label'] >= 'LABEL_1']
    NegDF = comment_DF.loc[comment_DF['label'] >= 'LABEL_0']
    
    
    
        
    Pos_text_list = PosDF['normalized_sents'].tolist()
    Neg_text_list = NegDF['normalized_sents'].tolist()

    
    
    
    def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
        output = []
        for sent in texts:
            doc = nlp(sent) 
            output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
        return output

    Pos_tokenized_reviews = lemmatization(Pos_text_list)
    Neg_tokenized_reviews = lemmatization(Neg_text_list)
    
    
    Pos_dictionary = corpora.Dictionary(Pos_tokenized_reviews)
    Neg_dictionary = corpora.Dictionary(Neg_tokenized_reviews)
    
    Pos_doc_term_matrix = [Pos_dictionary.doc2bow(word) for word in Pos_tokenized_reviews]
    Neg_doc_term_matrix = [Neg_dictionary.doc2bow(word) for word in Neg_tokenized_reviews]
    

    Poslda = LdaMulticore(corpus=Pos_doc_term_matrix, id2word=Pos_dictionary, num_topics=8, workers = 48)
    Neglda = LdaMulticore(corpus=Neg_doc_term_matrix, id2word=Neg_dictionary, num_topics=8, workers = 48)

    Postopics = Poslda.print_topics(num_words=10)
    Negtopics = Neglda.print_topics(num_words=10)
    
    PosdataList = list(Postopics)
    NegdataList = list(Negtopics)

    dfPos = pd.DataFrame(PosdataList, columns=['Topic', 'Words and probability'])
    dfNeg = pd.DataFrame(NegdataList, columns=['Topic', 'Words and probability'])
    

    dfPos.to_csv('Pos_LDA_Twitter%s.csv' %f)
    dfNeg.to_csv('Neg_LDA_Twitter%s.csv' %f)

Dec2021
